In [ ]:
import nltk
import nltk.translate.gleu_score as gleu

import numpy

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small", max_model_length=512)
#encoded_input = tokenizer(df['gold'].tolist()[0], max_model_length=512)

In [ ]:
import pandas as pd

df = pd.read_csv('../input/t5-res/pred_t5 (2).csv')

In [ ]:
from nltk import word_tokenize

preds = df['preds'].apply(lambda x: ' '.join(word_tokenize(x))).tolist()
gold = df['gold'].apply(lambda x: ' '.join(word_tokenize(tokenizer.decode(tokenizer.encode(x, truncation=True, max_length=512), skip_special_tokens=True)))).tolist()

In [ ]:
with open('m2preds.txt', 'a', encoding='utf-8') as f:
    f.write('\n'.join(preds))
        
with open('m2gold.txt', 'a', encoding='utf-8') as f:
    f.write('\n'.join(gold))

In [ ]:
import re

df2 = pd.read_csv('../input/noisy-news/test_t5.csv')
inps = df2.input.apply(lambda x: ' '.join(word_tokenize(tokenizer.decode(tokenizer.encode(x[9:], truncation=True, max_length=512), skip_special_tokens=True)))).tolist()
with open('m2inp.txt', 'a', encoding='utf-8') as f:
    for pr in gold:
        f.write('\n'.join(pr))

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download ru_core_news_sm

In [ ]:
!python '../input/convert-m2/edit-extraction-master/parallel_to_m2.py' -orig 'm2inp.txt' -cor 'm2gold.txt' -out 'gold.m2'

In [ ]:
mn = []
eqs = 0
mr = []
for g,p in zip(gold, preds):
    if len(g[0]) < len(p):
        mn.append(len(p) - len(g[0]))
    elif len(g[0]) == len(p):
        eqs += 1
    else:
        mr.append(len(g[0]) - len(p))

In [ ]:
i = 0
for g,p in zip(gold, preds):
    if len(p) < len(g[0]):
        gold[i][0] = g[0][:len(p)]
    i += 1
gleu.corpus_gleu(gold, preds)

In [ ]:
import re

df2 = pd.read_csv('../input/noisy-news/test_t5.csv')
df2['input'] = df2.input.apply(lambda x: re.sub('\n', ' ', x[9:]))

In [ ]:
gold[0]

In [ ]:
from tqdm import tqdm

#y_golds = []
#y_preds = []
mets = {'precision': [], 'recall': [], 'f-score': []}
for inp, g, pred in tqdm(zip(inps, gold, preds)):
    tok_inp = inp.split()
    tok_tar = g.split()
    tok_pred = pred.split()
    #y_gold = []
    #y_pred = []
    fp = 0
    tp = 0
    tn = 0
    fn = 0
    for j, toks in enumerate(zip(tok_inp, tok_tar, tok_pred)):
        if toks[0] != toks[1]:
            if toks[0] == toks[2]:
                fn += 1
            elif toks[1] != toks[2]:
                fp += 1
            else:
                tp +=1
            #y_gold.append(f'{toks[1]}_{j}')
        elif toks[0] != toks[2]:
            fp +=1
        else:
            tn += 1
            #y_pred.append(f'{toks[2]}_{j}')
    try:
        p = tp/(tp+fp)
        r = tp/(tp+fn)
        f = (1.0+0.25) * p * r / (0.25*p+r)
    except: 
        p = 0.0
        r = 0.0
        f = 0.0
    mets['precision'].append(p)
    mets['recall'].append(r)
    mets['f-score'].append(f)   
    #y_golds.append(y_gold)
    #y_preds.append(y_pred)

In [ ]:
!pip install pymystem3

In [ ]:
import pandas as pd
import re

df = pd.read_csv('../input/t5-res/pred_t5 (2).csv')
df2 = pd.read_csv('../input/noisy-news/test_t5.csv')
df = df.applymap(lambda x: re.sub('[\n\t ]+', ' ', x))
gold = df['gold'].tolist()
preds = df['preds'].tolist()
inputs = df2['input'].apply(lambda x: re.sub('[\n\t ]+', ' ', x[9:])).tolist()

In [ ]:
from pymystem3 import Mystem
from tqdm import tqdm

m = Mystem()

def find_gnd(inf):
    if 'жен' in inf:
        return 'жен'
    elif 'муж' in inf:
        return 'муж'
    elif 'сред' in inf:
        return 'сред'
    else:
        return ''
    
def find_num(inf):
    if 'ед' in inf:
        return 'ед'
    elif 'мн' in inf:
        return 'мн'
    else:
        return ''

def label_target(inps, tars):
    labels = []
    corrs = {}
    for k, (inp, tars) in tqdm(enumerate(zip(inps, tars))):
        toks_inp = m.analyze(inp.lower())
        toks_tar = m.analyze(tars.lower())
        can_pp = ''
        pp = ''
        text_tags = []
        pp_ind = 0
    #toks_pred = m.analyze(pred)
        for i, (t_i, t_t) in enumerate(zip(toks_inp, toks_tar)):
            #if pp != '':
             #   print(pp, pp_ind)
            if i - pp_ind > 10:
                #pp = ''
                can_pp = ''
            if can_pp + ' '* min(len(can_pp), 1) + t_i['text'].upper() in pps and not t_i['text'].isspace():
                #print(pp + ' '* min(len(pp), 1) + t_t['text'].upper())
                can_pp = can_pp + ' '* min(len(can_pp), 1) + t_t['text'].upper()
                pp_ind = i
            elif can_pp != '' and t_i['text'].upper() in pps and not t_i['text'].isspace():
                can_pp = t_i['text'].upper()
                pp_ind = i
            elif can_pp != '' and can_pp in pp_dict and not t_i['text'].isspace():
                pp_ind = i - 2
                pp = can_pp
                can_pp = ''
            if 'analysis' in t_i and 'analysis' in t_t and pp != '': 
                if len(t_i['analysis']) > 0 and len(t_t['analysis']) > 0:
                    if t_i['analysis'][0]['gr'].split('=')[0].split(',')[0] in ['S', 'SPRO', 'A', 'ANUM', 'APRO'] and t_i['analysis'][0]['lex'] == t_t['analysis'][0]['lex']:
                        an =  t_t['analysis'][0]['gr'].split('=')[1].split('|')
                        an_dict = {}
                        for a in an:
                        #a = a.split(',', 1)
                            an_dict[a.split(',')[0].strip('(').strip(')')] = {'gender': find_gnd(a), 'number': find_num(a)}
                        #except:
                         #   print(an)
                         #   an_dict[a[0]] = {'gender': find_gnd(a[1]), 'number': find_num(a[1])}
                        anc = list(an_dict.keys())
                        info =  t_t['analysis'][0]['gr'].split('=')[0]
                        pos = info.split(',')[0]
                        gnd = find_gnd(info)
                        nm = find_num(info)
                        case = ''
                        for cs in sorted(pp_dict[pp.upper()].keys(), key=lambda x: pp_dict[pp.upper()][x], reverse=True):
                            if cs in an_dict.keys():
                                case = cs
                                break
                        if case == '':
                            case = anc[0]
                        if case == 'им' and len(anc) > 1:
                            case = anc[1]
                        if gnd == '':
                            gnd = an_dict[case]['gender']
                        if nm == '':
                            nm = an_dict[case]['number']
                        text_tags.append('_'.join([pp, case, gnd, nm]))
                        corrs[f'{k}_{i}'] = {'prep': pp, 'case': case, 'gender': gnd, 'number': nm, 'lex': t_t['analysis'][0]['lex'], 'pos': pos}
                        if pp != '' and pos in ['S', 'SPRO']:
                            pp = ''
                    if t_i['text'] == t_t['text']:
                        text_tags.append('SAME')
                    else:
                        text_tags.append('OTHER')
                if t_i['text'] == t_t['text']:
                    text_tags.append('SAME')
                else:
                    text_tags.append('OTHER')
            elif t_i['text'] == t_t['text']:
                text_tags.append('SAME')
            else:
                text_tags.append('OTHER')
            #elif t_i['text'].isspace():
                #text_tags.append('SAME')
        labels.append(text_tags)
    return labels, corrs

In [121]:
import json

with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(cd4, f)

In [ ]:
model = [
["О",0.0, 0.0, 0.0769, 0.0, 0.923],
["В",0.1, 0.0, 0.276, 0.0, 0.712], # Добавлен gent
["ОБ",0.0, 0.0, 0.0222, 0.0, 0.978],
["ПО",0.1, 0.986, 0.00859, 0.0, 0.00515], # Добавлен gent
["БЕЗ",0.986, 0.0, 0.0, 0.0, 0.0],
["С",0.213, 0.0, 0.00242, 0.772, 0.0],
["НЕ В",0.0, 0.0, 0.288, 0.0, 0.712],
["ПРО",0.0, 0.0, 1.0, 0.0, 0.0],
["ДО",1.0, 0.0, 0.0, 0.0, 0.0],
["ПРИ",0.0, 0.0, 0.0, 0.0, 0.995],
["К",0.0, 1.0, 0.0, 0.0, 0.0],
["СО",0.321, 0.0, 1.0, 0.666, 0.0],
["НА",0.1, 0.0, 0.358, 0.0, 0.635], # Добавлен gent
["ВО",0.0, 0.0, 0.142, 0.0, 0.811],
["ЗА",0.0, 0.0, 0.358, 0.603, 0.0],
["ПОД",0.0, 0.0, 0.191, 0.779, 0.0],
["ПЕРЕД",0.0, 0.0, 0.0, 1.0, 0.0],
["ДЛЯ",1.0, 0.0, 0.0, 0.0, 0.0],
["ЧЕРЕЗ",0.0, 0.0, 1.0, 0.0, 0.0], # Добавлен accs
["ПОСЛЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["МЕЖДУ",0.156, 0.0, 0.0, 0.844, 0.0],
["НЕ БЕЗ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВО ВРЕМЯ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ХОДЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПРОТИВ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПРЕД",0.0, 0.0, 0.0, 1.0, 0.0],
["НАД",0.0, 0.0, 0.0, 1.0, 0.0],
["ВВИДУ",1.0, 0.0, 0.0, 0.0, 0.0],
["ОТНОСИТЕЛЬНО", 1.0, 0.0, 0.0, 0.0, 0.0],
["КО",0.0, 1.0, 0.0, 0.0, 0.0],
["СООТВЕТСТВЕННО", 0.0, 1.0, 0.0, 0.0, 0.0],
["ОТ",1.0, 0.0, 0.0, 0.0, 0.0],
["В РЕЗУЛЬТАТЕ", 0.973, 0.0, 0.0, 0.0, 0.0],
["В ТЕЧЕНИЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["С ПОМОЩЬЮ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПУТЕМ",1.0, 0.0, 0.0, 0.0, 0.0],
["СОГЛАСНО",0.0, 0.909, 0.0, 0.0, 0.0],
["В КАЧЕСТВЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ОКОЛО",1.0, 0.0, 0.0, 0.0, 0.0],
["БЛАГОДАРЯ",0.0, 1.0, 0.0, 0.0, 0.0],
["В ВИДЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["В СООТВЕТСТВИИ С", 0.0, 0.0, 0.0, 1.0, 0.0],
["ПРИ ПОМОЩИ",1.0, 0.0, 0.0, 0.0, 0.0],
["СРЕДИ",1.0, 0.0, 0.0, 0.0, 0.0],
["МИМО",1.0, 0.0, 0.0, 0.0, 0.0],
["СКВОЗЬ",0.0, 0.0, 1.0, 0.0, 0.0],
["ВДОЛЬ",1.0, 0.0, 0.0, 0.0, 0.0],
["МЕЖ",0.0, 0.0, 0.0, 1.0, 0.0],
["РЯДОМ С",0.0, 0.0, 0.0, 1.0, 0.0],
["В СТОРОНУ",0.857, 0.0, 0.0, 0.0, 0.0],
["ВМЕСТЕ С",0.0, 0.0, 0.0, 1.0, 0.0],
["ВПЕРЕДИ",0.75, 0.0, 0.0, 0.0, 0.0],
["ПО НАПРАВЛЕНИЮ К", 0.0, 1.0, 0.0, 0.0, 0.0],
["ВСЛЕД ЗА",0.0, 0.0, 0.0, 1.0, 0.0],
["ВОКРУГ",0.833, 0.0, 0.0, 0.0, 0.0],
["СЛЕДОМ ЗА",0.0, 0.0, 0.0, 1.0, 0.0],
["В НОГУ С",0.0, 0.0, 0.0, 1.0, 0.0],
["НАПЕРЕКОР",0.0, 1.0, 0.0, 0.0, 0.0],
["ВПЕРЕД",1.0, 0.0, 0.0, 0.0, 0.0],
["РАДИ",1.0, 0.0, 0.0, 0.0, 0.0],
["НА ОСНОВАНИИ", 1.0, 0.0, 0.0, 0.0, 0.0],
["С УЧЕТОМ",1.0, 0.0, 0.0, 0.0, 0.0],
["В СВЯЗИ С",0.0, 0.0, 0.0, 1.0, 0.0],
["ОТ ИМЕНИ",1.0, 0.0, 0.0, 0.0, 0.0],
["В НАЧАЛЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["У",0.991, 0.0, 0.0, 0.0, 0.0],
["НА ПОРОГЕ",0.833, 0.0, 0.0, 0.0, 0.0],
["ВНУТРИ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ПРОЦЕССЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["НЕДАЛЕКО ОТ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВБЛИЗИ",1.0, 0.0, 0.0, 0.0, 0.0],
["НА МЕСТЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВНЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["НИЖЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["НАПРОТИВ",1.0, 0.0, 0.0, 0.0, 0.0],
["ДАЛЕКО ЗА",0.0, 0.0, 0.0, 1.0, 0.0],
["В ПРЕДЕЛАХ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВОЗЛЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["В РАСПОРЯЖЕНИИ", 1.0, 0.0, 0.0, 0.0, 0.0],
["ВЫШЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПОСРЕДИ",1.0, 0.0, 0.0, 0.0, 0.0],
["НАКАНУНЕ",0.5, 0.0, 0.0, 0.0, 0.0],
["ПОСРЕДСТВОМ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВОПРЕКИ",0.0, 1.0, 0.0, 0.0, 0.0],
["ИЗО",1.0, 0.0, 0.0, 0.0, 0.0],
["НА СТРАЖЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПОСЕРЕДИНЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["В СВЕТЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["НЕПОДАЛЕКУ ОТ", 1.0, 0.0, 0.0, 0.0, 0.0],
["ПОСРЕДИНЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПОЗАДИ",0.5, 0.0, 0.0, 0.0, 0.0],
["ПОВЕРХ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВСЛЕД",0.0, 1.0, 0.0, 0.0, 0.0],
["ЛИЦОМ К ЛИЦУ С", 0.0, 0.0, 0.0, 1.0, 0.0],
["В СИЛУ",1.0, 0.0, 0.0, 0.0, 0.0],
["СОВМЕСТНО С",0.0, 0.0, 0.0, 1.0, 0.0],
["ЗА СЧЕТ",1.0, 0.0, 0.0, 0.0, 0.0],
["СПУСТЯ",0.0, 0.0, 1.0, 0.0, 0.0],
["НА БЛАГО",1.0, 0.0, 0.0, 0.0, 0.0],
["В ОБЛАСТИ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПО ОКОНЧАНИИ", 1.0, 0.0, 0.0, 0.0, 0.0],
["НЕ ЗА",0.0, 0.0, 1.0, 1.0, 0.0],
["ВНИЗУ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВМЕСТО",1.0, 0.0, 0.0, 0.0, 0.0],
["ПОПЕРЕК",1.0, 0.0, 0.0, 0.0, 0.0],
["ВНУТРЬ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ПАМЯТЬ О",0.0, 0.0, 0.0, 0.0, 1.0],
["ВПЛОТНУЮ К",0.0, 1.0, 0.0, 0.0, 0.0],
["НА ПРОТЯЖЕНИИ", 1.0, 0.0, 0.0, 0.0, 0.0],
["В СОГЛАСИИ С", 0.0, 0.0, 0.0, 1.0, 0.0],
["БЕЗО",1.0, 0.0, 0.0, 0.0, 0.0],
["ПО СЛУЧАЮ",1.0, 0.0, 0.0, 0.0, 0.0],
["СО СТОРОНЫ",1.0, 0.0, 0.0, 0.0, 0.0],
["ВСЛЕДСТВИЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["В РАСПОРЯЖЕНИЕ", 1.0, 0.0, 0.0, 0.0, 0.0],
["С ЦЕЛЬЮ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ШАГЕ ОТ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПО ОТНОШЕНИЮ К", 0.0, 1.0, 0.0, 0.0, 0.0],
["В РОЛИ",1.0, 0.0, 0.0, 0.0, 0.0],
["СООТВЕТСТВЕННО С", 0.0, 0.0, 0.0, 1.0, 0.0],
["СВЕРХУ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ИНТЕРЕСАХ",1.0, 0.0, 0.0, 0.0, 0.0],
["ПО МЕРЕ",1.0, 0.0, 0.0, 0.0, 0.0],
["НА СМЕНУ",0.0, 1.0, 0.0, 0.0, 0.0],
["ПО СРАВНЕНИЮ С", 0.0, 0.0, 0.0, 1.0, 0.0],
["ИСХОДЯ ИЗ",1.0, 0.0, 0.0, 0.0, 0.0],
["В ЗАВИСИМОСТИ ОТ", 1.0, 0.0, 0.0, 0.0, 0.0],
["ПО ПОВОДУ",1.0, 0.0, 0.0, 0.0, 0.0],
["ИЗНУТРИ",1.0, 0.0, 0.0, 0.0, 0.0]
]

pp_dict = {}
pps = []
for pp in model:
    pp_dict[pp[0]] = {}
    pps.append(' '.join(pp[0].split(' ')[:1]))
    pps.append(' '.join(pp[0].split(' ')[:2]))
    pps.append(' '.join(pp[0].split(' ')[:3]))
    pps.append(' '.join(pp[0].split(' ')[:4]))
    for i in zip(pp[1:], ['род', 'дат', 'вин', 'твор', 'пр']):
        if i[0] != 0.0:
            pp_dict[pp[0]][i[1]] = i[0]
pps = set(pps)

In [ ]:
gold_labels, gold_corrs = label_target(inputs, gold)

In [ ]:
pred_labels, pred_corrs = label_target(inputs, preds)

In [ ]:
import json

with open('gold_res_5.json', 'w', encoding='utf-8') as f:
    json.dump({'gold_labels': gold_labels, 'gold_corrs': gold_corrs, 'pred_labels': pred_labels, 'pred_corrs': pred_corrs}, f)

In [ ]:
import json

with open('../input/t5foranalysis/gold_res_5.json', 'r', encoding='utf-8') as f:
    res_dict = json.load(f)

In [ ]:
import pandas as pd
import re

df = pd.read_csv('../input/t5-res/pred_t5 (2).csv')

In [ ]:
df2 = pd.read_csv('../input/noisy-news/test_t5.csv')

In [ ]:
df['gold'][20]

In [ ]:
df2['input'][20]

In [ ]:
res_dict['pred_labels'][4]

In [ ]:
from tqdm import tqdm

y_pred = []
y_true = []
ps = []
gs = []
cases = ['род','дат','вин','твор','пр', 'парт', 'мест', 'SAME', 'OTHER']
def find_case(x):
    for t in x.split('_'):
        if t in cases:
            return t
    return 'OTHER'
for pr, gl in tqdm(zip(res_dict['pred_labels'], res_dict['gold_labels'])):
    for p, g in zip(pr, gl):
        #if not (p == g and g =='SAME'):
        ps.append(find_case(p))
        gs.append(find_case(g))
    
    #y_pred.append(gs)
    #y_true.append(ps)

In [ ]:
import sklearn
sklearn.metrics.classification_report(gs,ps, output_dict=True)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
#labs = set(y_pred)
cm = confusion_matrix(gs, ps, labels=list(set(ps)), normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=set(ps))
#figure, ax = plt.plot(figsize=(10,10))
disp.plot()
plt.show()

In [ ]:
from tqdm import tqdm


pls = []
gls = []
cases = ['род','дат','вин','твор','пр', 'SAME', 'OTHER']
def label_case(x):
    lb = ''
    for t in x.split('_'):
        if t in pp_dict:
            lb = t
        if t in cases and lb != '':
            return f'{lb}_{t}'
    
    return f'{lb}_OTHER'
        
for pr, gl in tqdm(zip(res_dict['pred_labels'], res_dict['gold_labels'])):
    for p, g in zip(pr, gl):
        ps = l_case(p))
        gs.append(find_case(g))
    
    #y_pred.append(gs)
    #y_true.append(ps)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,10)
#labs = set(y_pred)
cm = confusion_matrix(gs, ps, labels=['им','род','дат','вин','твор','пр', 'SAME', 'OTHER'], normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['им','род','дат','вин','твор','пр', 'SAME', 'OTHER'])
#figure, ax = plt.plot(figsize=(10,10))
disp.plot()
plt.show()

In [ ]:
import sklearn
sklearn.metrics.classification_report(gs,ps, output_dict=True)

In [ ]:
from sklearn.metrics import classification_report
classification_report(gs,ps, output_dict=True, labels=['им','род','дат','вин','твор','пр',])

In [ ]:
from sklearn.metrics import fbeta_score

fbeta_score(gs,ps,beta=0.5,labels=['род','дат','вин','твор','пр'], average='weighted')

In [ ]:
sum(mets['precision'])/len(mets['precision'])

In [ ]:
sum(mets['recall'])/len(mets['recall'])

In [ ]:
sum(mets['f-score'])/len(mets['f-score'])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_golds[2], y_preds[2], average='micro', beta=0.5)

In [ ]:
y_preds[0]

In [ ]:
y_golds[0]

In [ ]:
gleu.corpus_gleu(gold, preds)

In [ ]:
len(max(preds, key=lambda x: len(x)))

In [ ]:
len(max(gold, key=lambda x: len(x[0]))[0])

In [ ]:
import shutil

shutil.copyfile('../input/t5-res/m2scorer/scripts/stats.txt', 'stats.txt')

In [ ]:
!python '../input/t5-res/m2scorer/scripts/m2scorer.py' '../input/t5-res/preds.txt' 't5gold2.txt'

In [147]:
    pp_dict['ЗА']

In [148]:
for k,v in cd4['пр'].items():
    print(k, v['all'])

In [145]:
955/(2817 + 955 + 641 + 17)

In [149]:
cd4['пр']

In [134]:
1865/(3136+1865+8+2)